In [1]:
import time

notebook_start_time = time.time()

In [2]:
import sys
from pathlib import Path   
root_dir = str(Path().absolute().parent)

if root_dir not in sys.path:
    print(f"Adding the following directory to the PYTHONPATH: {root_dir}")
    sys.path.append(root_dir)

Adding the following directory to the PYTHONPATH: d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system


In [3]:
from recsys.config import settings
from pprint import pprint

d:\Projects\ML\personalized-recommendation-sysyem\personalized-recommendation-system\uvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
pprint(dict(settings))

{'CUSTOMER_DATA_SIZE': <CustomerDatasetSize.SMALL: 'SMALL'>,
 'CUSTOM_HOPSWORKS_INFERENCE_ENV': 'custom_env_name',
 'FEATURES_EMBEDDING_MODEL_ID': 'all-MiniLM-L6-v2',
 'HOPSWORKS_API_KEY': SecretStr('**********'),
 'OPENAI_API_KEY': SecretStr(''),
 'OPENAI_MODEL_ID': 'gpt-4o-mini',
 'RANKING_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'RANKING_EARLY_STOPPING_ROUNDS': 5,
 'RANKING_ITERATIONS': 100,
 'RANKING_LEARNING_RATE': 0.2,
 'RANKING_MODEL_TYPE': 'ranking',
 'RANKING_SCALE_POS_WEIGHT': 10,
 'RECSYS_DIR': WindowsPath('d:/Projects/ML/personalized-recommendation-sysyem/personalized-recommendation-system/recsys'),
 'TWO_TOWER_DATASET_TEST_SPLIT_SIZE': 0.1,
 'TWO_TOWER_DATASET_VALIDATON_SPLIT_SIZE': 0.1,
 'TWO_TOWER_LEARNING_RATE': 0.001,
 'TWO_TOWER_MODEL_BATCH_SIZE': 2048,
 'TWO_TOWER_MODEL_EMBEDDING_SIZE': 16,
 'TWO_TOWER_NUM_EPOCHS': 5,
 'TWO_TOWER_WEIGHT_DECAY': 0.001}


In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import warnings

warnings.filterwarnings("ignore")

from loguru import logger

from recsys import hopsworks_integration

In [7]:
project, fs = hopsworks_integration.get_feature_store()

2025-03-17 00:14:25.164 | INFO     | recsys.hopsworks_integration.feature_store:get_feature_store:13 - Loging to Hopsworks using HOPSWORKS_API_KEY env var.


2025-03-17 00:14:25,164 INFO: Initializing external client
2025-03-17 00:14:25,164 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-17 00:14:28,318 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659


Deploying the ranking inference pipeline

In [9]:
ranking_deployment = hopsworks_integration.ranking_serving.HopsworksRankingModel.deploy(
    project=project
)

Uploading: 100.000%|██████████| 4633/4633 elapsed<00:02 remaining<00:00
Uploading: 100.000%|██████████| 1149/1149 elapsed<00:01 remaining<00:00


Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1213659/deployments/361479
Before making predictions, start the deployment by using `.start()`


In [52]:
ranking_deployment.start()

Deployment is ready: 100%|██████████| 6/6 [00:33<00:00,  5.51s/it]

Start making predictions by using `.predict()`


In [19]:
# Check logs in case of failure
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1213659/deployments/361476

DeployableComponentLogs(instance_name: 'ranking-transformer-00001-deployment-56997df9d6-ww4md', date: datetime.datetime(2025, 3, 16, 19, 55, 9, 996228)) 
INFO:root:Loading component module...
INFO:root:[TransformerModel] Initializing transformer for model: ranking
INFO:root:[HopsworksModel] Initializing for model: ranking
INFO:hsfs.engine.python:Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai/p/1213659
... execution time: 10.983202 seconds
INFO:root:Starting KServe server...
2025-03-16 14:24:15.228 8 kserve INFO [model_server.py:register_model():363] Registering model: ranking
2025-03-16 14:24:15.228 8 kserve INFO [model_server.py:start():298] Setting max asyncio worker threads as 12
2025-03-16 14:24:15.228 8 kserve INFO [model_server.py:_serve_rest():244] Starting uvicorn with 1 workers


2025-03-16 14:24:15.321 uvicorn.error 

In [18]:
def get_top_recommendations(ranked_candidates, k=3):
    return [candidate[-1] for candidate in ranked_candidates["ranking"][:k]]

In [21]:
test_ranking_input = [
        {
            "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
            "month_sin": 1.2246467991473532e-16,
            "query_emb": [
                0.214135289,
                0.571055949,
                0.330709577,
                -0.225899458,
                -0.308674961,
                -0.0115124583,
                0.0730511621,
                -0.495835781,
                0.625569344,
                -0.0438038409,
                0.263472944,
                -0.58485353,
                -0.307070434,
                0.0414443575,
                -0.321789205,
                0.966559,
            ],
            "month_cos": -1.0,
        }
    ]

# Test ranking deployment
ranked_candidates = ranking_deployment.predict(inputs=test_ranking_input)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [24]:
ranking_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1213659/deployments/361476

DeployableComponentLogs(instance_name: 'ranking-transformer-00001-deployment-56997df9d6-6kbnc', date: datetime.datetime(2025, 3, 16, 19, 58, 10, 965935)) 
INFO:root:Loading component module...
INFO:root:[TransformerModel] Initializing transformer for model: ranking
INFO:root:[HopsworksModel] Initializing for model: ranking
INFO:hsfs.engine.python:Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai/p/1213659
... execution time: 10.908188 seconds
INFO:root:Starting KServe server...
2025-03-16 14:26:36.668 8 kserve INFO [model_server.py:register_model():363] Registering model: ranking
2025-03-16 14:26:36.668 8 kserve INFO [model_server.py:start():298] Setting max asyncio worker threads as 12
2025-03-16 14:26:36.668 8 kserve INFO [model_server.py:_serve_rest():244] Starting uvicorn with 1 workers


2025-03-16 14:26:36.721 uvicorn.error

Deploying the query inference pipeline

In [10]:
query_model_deployment = (
    hopsworks_integration.two_tower_serving.HopsworksQueryModel.deploy(ranking_model_type="ranking")
)

2025-03-17 00:16:21,636 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-17 00:16:21,645 INFO: Initializing external client
2025-03-17 00:16:21,645 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-17 00:16:24,471 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659
2025-03-17 00:16:27,501 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-17 00:16:27,501 INFO: Initializing external client
2025-03-17 00:16:27,501 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-17 00:16:30,365 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213659
Secret created successfully, explore it at https://c.app.hopsworks.ai:443/account/secrets


Uploading: 100.000%|██████████| 3050/3050 elapsed<00:02 remaining<00:00


Deployment created, explore it at https://c.app.hopsworks.ai:443/p/1213659/deployments/361480
Before making predictions, start the deployment by using `.start()`


In [48]:
query_model_deployment.start()

Deployment is ready: 100%|██████████| 6/6 [00:38<00:00,  6.45s/it]

Start making predictions by using `.predict()`


In [ ]:
# Check logs in case of failure
query_model_deployment.get_logs(component="transformer", tail=20)

Testing the inference pipeline

In [11]:
data = [
    {
        "customer_id": "d327d0ad9e30085a436933dfbb7f77cf42e38447993a078ed35d93e3fd350ecf",
        "transaction_date": "2022-11-15T12:16:25.330916",
    }
]

In [20]:
ranked_candidates = query_model_deployment.predict(inputs=data)

# Retrieve article ids of the top recommended items
recommendations = get_top_recommendations(ranked_candidates["predictions"], k=3)
recommendations

RestAPIError: Metadata operation error: (url: http://15.235.46.163/v1/models/query:predict). Server response: 
HTTP code: 500, HTTP reason: Internal Server Error, body: b'{"error":"HTTPError : HTTP 500: \'month_sin\'"}', error code: , error msg: , user msg: 

 Check the model server logs by using `.get_logs()`

In [ ]:
query_model_deployment.get_logs(component="transformer", tail=200)

Explore all the logs and filters in the Kibana logs at https://c.app.hopsworks.ai:443/p/1213659/deployments/361477



RestAPIError: Metadata operation error: (url: https://c.app.hopsworks.ai/hopsworks-api/api/project/1213659/serving/361477/logs). Server response: 
HTTP code: 404, HTTP reason: Not Found, body: b'{"errorCode":240027,"errorMsg":"Server logs not available"}', error code: 240027, error msg: Server logs not available, user msg: 

In [46]:
ranking_deployment.stop()
query_model_deployment.stop()

Deployment is stopped: 100%|██████████| 3/3 [00:11<00:00,  3.75s/it]
